In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix

# Load your data
df = pd.read_csv('../processed_data/merged_data_2017-2021.csv')
df = df.drop('Unnamed: 0', axis=1)  # Remove index column

# Basic exploration
print("Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"Counties: {df['county'].nunique()}")
print(f"Years: {df['year'].min()}-{df['year'].max()}")
print("\nMissing values:")
print(df.isnull().sum())

Dataset Overview:
Shape: (265, 19)
Counties: 53
Years: 2017-2021

Missing values:
county                                 0
year                                   0
days_with_aqi                          0
good_days                              0
moderate_days                          0
unhealthy_for_sensitive_groups_days    0
unhealthy_days                         0
very_unhealthy_days                    0
hazardous_days                         0
max_aqi                                0
90th_percentile_aqi                    0
median_aqi                             0
days_co                                0
days_no2                               0
days_ozone                             0
days_pm2.5                             0
days_pm10                              0
asthma_rate                            0
number_of_cases                        0
dtype: int64


In [ ]:
df = df.rename(columns=lambda c: c.replace('.', '_'))


In [ ]:
# Create meaningful features
df['total_unhealthy_days'] = (df['unhealthy_for_sensitive_groups_days'] + 
                              df['unhealthy_days'] + 
                              df['very_unhealthy_days'] + 
                              df['hazardous_days'])

df['percent_good_days'] = (df['good_days'] / df['days_with_aqi']) * 100
df['percent_unhealthy_days'] = (df['total_unhealthy_days'] / df['days_with_aqi']) * 100

# Air quality severity categories
df['aqi_category'] = pd.cut(df['median_aqi'], 
                           bins=[0, 50, 100, 150, 200, float('inf')],
                           labels=['Good', 'Moderate', 'Unhealthy for Sensitive', 'Unhealthy', 'Very Unhealthy'])

# High asthma rate binary target (for logistic regression)
asthma_threshold = df['asthma_rate'].quantile(0.75)  # Top 25% of asthma rate per 10,000
df['high_asthma'] = (df['asthma_rate'] > asthma_threshold).astype(int)

print(f"High asthma threshold: {asthma_threshold:.1f} cases per 10k")
print(f"Counties with high asthma rates: {df['high_asthma'].sum()}")


In [ ]:
# 1. Which air quality metrics correlate strongest with asthma?
air_quality_cols = ['median_aqi', 'max_aqi', '90th_percentile_aqi', 
                   'total_unhealthy_days', 'percent_unhealthy_days']

correlations = df[air_quality_cols + ['asthma_rate']].corr()['asthma_rate'].sort_values(ascending=False)
print("Correlations with asthma rate:")
print(correlations)

# 2. Pollutant-specific analysis
pollutant_days = ['days_co', 'days_no2', 'days_ozone', 'days_pm2_5', 'days_pm10'] #######
pollutant_corr = df[pollutant_days + ['asthma_rate']].corr()['asthma_rate'].sort_values(ascending=False)
print("\nPollutant correlations:")
print(pollutant_corr)

# 3. County and temporal patterns
county_stats = df.groupby('county').agg({
    'median_aqi': 'mean',
    'asthma_rate': 'mean',
    'total_unhealthy_days': 'mean'
}).sort_values('asthma_rate', ascending=False)

print("\nTop 10 counties by asthma rate:")
print(county_stats.head(10))

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

# Predict continuous asthma rate
features = ['median_aqi', 'max_aqi', 'total_unhealthy_days', 
           'days_pm2_5', 'days_ozone', 'percent_unhealthy_days'] ############

X = df[features]
y = df['asthma_rate']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Multiple Linear Regression:

# Train model
lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

# Predictions
y_pred = lr_model.predict(X_test_scaled)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"Linear Regression R²: {r2:.3f}")
print(f"RMSE: {rmse:.2f}")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': features,
    'coefficient': lr_model.coef_,
    'abs_coefficient': np.abs(lr_model.coef_)
}).sort_values('abs_coefficient', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

y_pred_rf = rf_model.predict(X_test)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest R²: {r2_rf:.3f}")

# Feature importance
rf_importance = pd.DataFrame({
    'feature': features,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nRandom Forest Feature Importance:")
print(rf_importance)

In [3]:
import statsmodels.formula.api as smf
mod = smf.ols('asthma_rate ~ median_aqi + C(county) + C(year)', data=df).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            asthma_rate   R-squared:                       0.903
Model:                            OLS   Adj. R-squared:                  0.876
Method:                 Least Squares   F-statistic:                     33.67
Date:                Mon, 23 Jun 2025   Prob (F-statistic):           1.79e-78
Time:                        02:26:12   Log-Likelihood:                -799.64
No. Observations:                 265   AIC:                             1715.
Df Residuals:                     207   BIC:                             1923.
Df Model:                          57                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       41.2006      5.579      7.385      0.000      30.201      52.200
C(county)[T.Amador]              5.1747      3.798      1.362      0.175      -2.314      12.663
C(county)[T.Butte]              -6.4251      3.542     -1.814      0.071     -13.409       0.558
C(county)[T.Calaveras]           6.1582      3.551      1.734      0.084      -0.842      13.159
C(county)[T.Colusa]             -2.4923      3.615     -0.690      0.491      -9.619       4.634
C(county)[T.Contra Costa]        5.3108      3.557      1.493      0.137      -1.703      12.324
C(county)[T.Del Norte]          11.7431      4.042      2.906      0.004       3.775      19.711
C(county)[T.El Dorado]          -8.0552      3.646     -2.209      0.028     -15.244      -0.866
C(county)[T.Fresno]             10.7616      3.881      2.773      0.006       3.109      18.414
C(county)[T.Glenn]              -4.2266      3.664     -1.153      0.250     -11.451       2.998
C(county)[T.Humboldt]            9.9493      4.050      2.456      0.015       1.964      17.934
C(county)[T.Imperial]            4.4463      3.615      1.230      0.220      -2.680      11.573
C(county)[T.Inyo]                8.0147      3.540      2.264      0.025       1.035      14.995
C(county)[T.Kern]               -6.2410      4.306     -1.449      0.149     -14.730       2.248
C(county)[T.Kings]               2.8608      3.731      0.767      0.444      -4.495      10.217
C(county)[T.Lake]               31.8184      3.999      7.956      0.000      23.934      39.703
C(county)[T.Los Angeles]        -6.1734      4.187     -1.474      0.142     -14.429       2.082
C(county)[T.Madera]              3.2964      3.570      0.923      0.357      -3.741      10.334
C(county)[T.Marin]             -18.6063      3.755     -4.956      0.000     -26.009     -11.204
C(county)[T.Mariposa]           -5.7522      3.547     -1.622      0.106     -12.745       1.241
C(county)[T.Mendocino]          14.4816      3.674      3.942      0.000       7.239      21.724
C(county)[T.Merced]             12.6949      3.542      3.584      0.000       5.711      19.678
C(county)[T.Mono]                3.7674      4.059      0.928      0.354      -4.235      11.770
C(county)[T.Monterey]           -6.1208      3.731     -1.640      0.102     -13.477       1.235
C(county)[T.Napa]               -7.9788      3.660     -2.180      0.030     -15.194      -0.764
C(county)[T.Nevada]            -12.6874      3.588     -3.536      0.001     -19.762      -5.613
C(county)[T.Orange]            -17.9732      3.543     -5.073      0.000     -24.958     -10.988
C(county)[T.Placer]            -13.6203      3.546     -3.841      0.000     -20.611      -6.630
C(county)[T.Plumas]              9.3484      3.552      2.632      0.009    

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

X = df[['median_aqi','year']]
y = df['asthma_rate']
rf = RandomForestRegressor(random_state=0)
scores = cross_val_score(rf, X, y, cv=5, scoring='r2')
scores.mean()

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# 1. Load and prepare your data
df = pd.read_csv('../processed_data/merged_data_2017-2021.csv')
X = pd.get_dummies(df[['median_aqi', 'county', 'year']], drop_first=True)
y = df['asthma_rate']

# 2. Out-of-sample test (train/test split)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

# Compute RMSE manually for compatibility
mse_test = mean_squared_error(y_test, preds)
rmse_test = np.sqrt(mse_test)
print(f"Test RMSE: {rmse_test:.3f}")

# 3. K-fold cross-validation (5 folds)
# Use negative MSE scoring, then convert to RMSE
cv_neg_mse = cross_val_score(
    LinearRegression(), X, y,
    cv=5, scoring='neg_mean_squared_error'
)
mse_cv = -cv_neg_mse
rmse_cv = np.sqrt(mse_cv)
print(f"CV RMSE scores: {rmse_cv.round(3)}")
print(f"Mean CV RMSE: {rmse_cv.mean():.3f}")

Test RMSE: 6.971
CV RMSE scores: [ 9.248  5.988 10.36   9.469  7.749]
Mean CV RMSE: 8.563


In [7]:
import numpy as np
base_pred = np.full_like(y_test, y_train.mean())
baseline_rmse = np.sqrt(mean_squared_error(y_test, base_pred))
print("Baseline RMSE:", baseline_rmse)

Baseline RMSE: 15.938285503297426


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.metrics import mean_squared_error

# 1. Load and prepare the data (adjust path if needed)
df = pd.read_csv('../processed_data/merged_data_2017-2021.csv')
X = pd.get_dummies(df[['median_aqi', 'county', 'year']], drop_first=True)
y = df['asthma_rate']

# 2. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Ridge Regression with cross-validated alpha
alphas = np.logspace(-3, 3, 13)
ridge_cv = RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X_train, y_train)
ridge_preds = ridge_cv.predict(X_test)
rmse_ridge = np.sqrt(mean_squared_error(y_test, ridge_preds))

# 4. Lasso Regression with cross-validated alpha
lasso_cv = LassoCV(cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
lasso_preds = lasso_cv.predict(X_test)
rmse_lasso = np.sqrt(mean_squared_error(y_test, lasso_preds))

# 5. Output results
print(f"Ridge best alpha: {ridge_cv.alpha_}")
print(f"Ridge Test RMSE: {rmse_ridge:.3f}")
print(f"Lasso best alpha: {lasso_cv.alpha_:.5f}")
print(f"Lasso Test RMSE: {rmse_lasso:.3f}")

Ridge best alpha: 0.31622776601683794
Ridge Test RMSE: 6.977
Lasso best alpha: 0.01468
Lasso Test RMSE: 7.044


In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

rf = RandomForestRegressor(random_state=42)
cv_neg_mse = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_squared_error')
rmse_rf = np.sqrt(-cv_neg_mse)
print("RF CV RMSE:", rmse_rf, "Mean:", rmse_rf.mean())

RF CV RMSE: [10.07924253  6.51853301  8.28152172 19.629772    4.45830923] Mean: 9.793475697975321


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

# define your features X and target y as before
rf = RandomForestRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_rf = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_rf.fit(X, y)
best_rmse = np.sqrt(-grid_rf.best_score_)
print("Best RF params:", grid_rf.best_params_)
print(f"Best RF CV RMSE: {best_rmse:.3f}")

Best RF params: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Best RF CV RMSE: 11.089


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, GroupKFold
import numpy as np

# Prepare X, y as before; and groups = df['county'] if you want county‐aware splits
gbm = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [1, 3, 5]
}

# Optional: use GroupKFold to ensure each county appears in every fold
gkf = GroupKFold(n_splits=5)
grid_gbm = GridSearchCV(
    gbm,
    param_grid,
    cv=gkf.split(X, y, groups=df['county']),
    scoring='neg_mean_squared_error',
    n_jobs=-1
)
grid_gbm.fit(X, y)

best_rmse = np.sqrt(-grid_gbm.best_score_)
print("Best GBM params:", grid_gbm.best_params_)
print(f"Best GBM CV RMSE: {best_rmse:.3f}")

Best GBM params: {'learning_rate': 0.1, 'max_depth': 3, 'min_samples_leaf': 1, 'n_estimators': 500}
Best GBM CV RMSE: 12.389
